# Travelio Challenge


Data Preparation

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
dataset_path = "CHALLNGE_DATA-SET/"

# Parameters
img_height, img_width = 224, 224  # Consistent size for all images
batch_size = 32

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Split data into training and validation sets
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 302 images belonging to 7 classes.
Found 73 images belonging to 7 classes.


# Model Creation

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load the VGG16 model with pre-trained weights, exclude top layers
base_model = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Create the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


# Training and Validation

In [3]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10  # You can increase the number of epochs
)


Epoch 1/10


C:\Users\doem1\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 114s 7s/step - accuracy: 0.2364 - loss: 5.3934 - val_accuracy: 0.6301 - val_loss: 1.0923
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 106s 5s/step - accuracy: 0.6262 - loss: 1.5277 - val_accuracy: 0.8356 - val_loss: 0.4633
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 107s 6s/step - accuracy: 0.8223 - loss: 0.4763 - val_accuracy: 0.8493 - val_loss: 0.4408
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 102s 7s/step - accuracy: 0.8577 - loss: 0.3222 - val_accuracy: 0.8767 - val_loss: 0.3330
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 84s 5s/step - accuracy: 0.9130 - loss: 0.2315 - val_accuracy: 0.9178 - val_loss: 0.2844
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 111s 6s/step - accuracy: 0.9412 - loss: 0.1832 - val_accuracy: 0.8630 - val_loss: 0.4060
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.9480 - loss: 0.1300 - val_accuracy: 0.9178 - val_loss: 0.2484
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step - accuracy: 0.9364 - loss: 0.1421 - val_accuracy: 0.9041 - val_loss: 0.308

#  Save the Model

In [5]:
model.save('attraction_model.h5')

# Load and Use the Model for Prediction

Training

In [11]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the model
model = load_model('attraction_model.h5')

# Function to predict the class of a single image
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_label = list(train_generator.class_indices.keys())[class_idx]

    return class_label

# Predict an image
img_path = 'E:\downloads\WhatsApp Image 2024-06-26 at 20.42.50_0edd5e07.jpg'
predicted_class = predict_image(img_path, model)
print(f'The image is predicted to be: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
The image is predicted to be: KHALIFA_TOWER
